# 1️⃣ Training an Adapter for a Transformer model

In this notebook, we train an adapter for a **RoBERTa** ([Liu et al., 2019](https://arxiv.org/pdf/1907.11692.pdf)) model for sequence classification on a **sentiment analysis** task using [adapter-transformers](https://github.com/Adapter-Hub/adapter-transformers), the _AdapterHub_ adaptation of HuggingFace's _transformers_ library.

If you're unfamiliar with the theoretical parts of adapters or the AdapterHub framework, check out our [introductory blog post](https://adapterhub.ml/blog/2020/11/adapting-transformers-with-adapterhub/) first.

We train a **Task Adapter** for a pre-trained model here. Most of the code is identical to a full finetuning setup using HuggingFace's transformers. For comparison, have a look at the [same guide using full finetuning](https://colab.research.google.com/drive/1brXJg5Mokm8h3shxqPRnoIsRwHQoncus?usp=sharing).

For training, we use the [movie review dataset by Pang and Lee (2005)](http://www.cs.cornell.edu/people/pabo/movie-review-data/). It contains movie reviews  from Rotten Tomatoes which are either classified as positive or negative. We download the dataset via HuggingFace's [datasets](https://github.com/huggingface/datasets) library.

## Installation

First, let's install the required libraries:

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -U adapter-transformers
!pip install datasets

     |████████████████████████████████| 2.9 MB 10.2 MB/s 
     |████████████████████████████████| 3.3 MB 65.1 MB/s 
     |████████████████████████████████| 596 kB 73.6 MB/s 
     |████████████████████████████████| 895 kB 76.3 MB/s 
     |████████████████████████████████| 61 kB 644 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 298 kB 6.7 MB/s 
     |████████████████████████████████| 132 kB 77.5 MB/s 
     |████████████████████████████████| 243 kB 77.2 MB/s 
     |████████████████████████████████| 1.1 MB 52.3 MB/s 
     |████████████████████████████████| 192 kB 59.4 MB/s 
     |████████████████████████████████| 160 kB 77.8 MB/s 
     |████████████████████████████████| 271 kB 90.7 MB/s 


## Dataset Preprocessing

Before we start to train our adapter, we first prepare the training data. Our training dataset can be loaded via HuggingFace `datasets` using one line of code:

In [3]:
from datasets import load_dataset, DatasetDict

dataset = load_dataset("yxchar/sciie-tlm")
dataset.num_rows

Using custom data configuration yxchar___sciie-tlm-a32f1f2c4e9b5c0d


  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/663k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/199k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/yxchar___sciie-tlm-a32f1f2c4e9b5c0d/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

{'test': 974, 'train': 3219, 'validation': 455}

Every dataset sample has an input text and a binary label:

In [4]:
max([len(x['text']) for x in dataset['train']])

577

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'id', 'label'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'id', 'label'],
        num_rows: 974
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'text', 'id', 'label'],
        num_rows: 455
    })
})

Now, we need to encode all dataset samples to valid inputs for our Transformer model. Since we want to train on `roberta-base`, we load the corresponding `RobertaTokenizer`. Using `dataset.map()`, we can pass the full dataset through the tokenizer in batches:

In [6]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=512, truncation=True, padding="max_length")

# Encode the input data
dataset = dataset.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset.rename_column_("label", "labels")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: rename_column_ is deprecated and will be removed in the next major version of datasets. Use DatasetDict.rename_column instead.
  if sys.path[0] == '':


Now we're ready to train our model...

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'attention_mask', 'id', 'input_ids', 'labels', 'text'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['Unnamed: 0', 'attention_mask', 'id', 'input_ids', 'labels', 'text'],
        num_rows: 974
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'attention_mask', 'id', 'input_ids', 'labels', 'text'],
        num_rows: 455
    })
})

## Training

We use a pre-trained RoBERTa model from HuggingFace. We use `RobertaModelWithHeads`, a class unique to `adapter-transformers`, which allows us to add and configure prediction heads in a flexibler way.

In [8]:
import numpy as np

In [9]:
nLabels = len(np.unique(dataset['train']['labels']))
nLabels

7

In [10]:
from transformers import RobertaConfig, RobertaModelWithHeads

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=nLabels,
)
model = RobertaModelWithHeads.from_pretrained(
    "roberta-base",
    config=config,
)

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

**Here comes the important part!**

We add a new adapter to our model by calling `add_adapter()`. We pass a name (`"rotten_tomatoes"`) and [the type of adapter](https://docs.adapterhub.ml/adapters.html#adapter-types) (task adapter). Next, we add a binary classification head. It's convenient to give the prediction head the same name as the adapter. This allows us to activate both together in the next step. The `train_adapter()` method does two things:

1. It freezes all weights of the pre-trained model so only the adapter weights are updated during training.
2. It activates the adapter and the prediction head such that both are used in every forward pass.

In [11]:
adapter_name = model.load_adapter("/content/drive/MyDrive/swp/SCIERC Results/final_adapter")

In [ ]:
# # Add a new adapter
# model.add_adapter("sciie")
# # Add a matching classification head
# model.add_classification_head(
#     "sciie",
#     num_labels=7,
#     # id2label={ 0: "👎", 1: "👍"}
#   )

# # Activate the adapter
# model.train_adapter("sciie")

In [12]:
model.set_active_adapters(adapter_name)

In [13]:
import torch
device='cuda'

In [14]:
# test_dataset[0]

In [15]:
test_dataset = dataset['test']
bsz = 16
# bsz = test_dataset.num_rows
i = 0
batches = []
while i<test_dataset.num_rows:
  batches.append(test_dataset[i:i+bsz])
  i+=bsz

  

In [16]:
# Put model in evaluation mode
model.to(device)
model.eval()


# Tracking variables for storing ground truth and predictions 
predictions , true_labels = [], []

# Prediction Loop
for i,batch in enumerate(batches):
  print(i)

 
 
  # Unpack the inputs from our dataloader and move to GPU/accelerator 
 
  input_ids = batch['input_ids'].to(device)
  attention_mask = batch['attention_mask'].to(device)
  labels = batch['labels'].to(device)

  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(input_ids, attention_mask=attention_mask, 
                         labels=labels)

  logits = outputs[1]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60


In [ ]:
# import matplotlib.pyplot as plt

In [ ]:
# fig, a = plt.subplots(2,3)
# a[0,0].hist(logits[:,0])
# a[0,1].hist(logits[:,1])
# a[0,2].hist(logits[:,2])
# a[1,0].hist(logits[:,3])
# a[1,1].hist(logits[:,4])
# a[1,2].hist(logits[:,5])

In [40]:
labels = np.concatenate(true_labels)
prds = np.concatenate(predictions).argmax(axis=1)

In [33]:
from sklearn.metrics import f1_score, classification_report

In [42]:
f1_score(labels, prds, average='micro')

0.8480492813141683

In [43]:
print(classification_report(labels, prds, labels=list(range(nLabels)), digits=4))

              precision    recall  f1-score   support

           0     0.8931    0.9512    0.9213       123
           1     0.6071    0.5763    0.5913        59
           2     0.8108    0.8955    0.8511        67
           3     0.9297    0.8931    0.9110       533
           4     0.5161    0.5079    0.5120        63
           5     0.7273    0.8421    0.7805        38
           6     0.7895    0.8242    0.8065        91

    accuracy                         0.8480       974
   macro avg     0.7534    0.7843    0.7677       974
weighted avg     0.8496    0.8480    0.8481       974



In [ ]:
sum(dataset['train']['labels']==5)

tensor(60)

In [ ]:
# from datasets import list_metrics
# metrics_list = list_metrics()
# # len(metrics_list)

# print(', '.join(metric for metric in metrics_list))

For training, we make use of the `Trainer` class built-in into `transformers`. We configure the training process using a `TrainingArguments` object and define a method that will calculate the evaluation accuracy in the end. We pass both, together with the training and validation split of our dataset, to the trainer instance.

**Note the differences in hyperparameters compared to full finetuning.** Adapter training usually required a few more training epochs than full finetuning.

In [ ]:
# import torch, gc

# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
# import numpy as np
# from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

# training_args = TrainingArguments(
#     learning_rate=2*1e-5,
#     num_train_epochs=30,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     logging_steps=200,
#     output_dir="./training_output",
#     overwrite_output_dir=True,
#     # The next line is important to ensure the dataset labels are properly passed to the model
#     remove_unused_columns=False,
# )

# def compute_accuracy(p: EvalPrediction):
#   preds = np.argmax(p.predictions, axis=1)
#   return {"acc": (preds == p.label_ids).mean()}

# trainer = AdapterTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=dataset["train"],
#     eval_dataset=dataset["validation"],
#     compute_metrics=compute_accuracy,
# )

Start the training 🚀

In [ ]:
# trainer.train()

Looks good! Let's evaluate our adapter on the validation split of the dataset to see how well it learned:

In [ ]:
# trainer.evaluate()

We can put our trained model into a `transformers` pipeline to be able to make new predictions conveniently:

In [ ]:
# from transformers import TextClassificationPipeline

# classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=training_args.device.index)

# classifier("This is awesome!")

At last, we can also extract the adapter from our model and separately save it for later reuse. Note the size difference compared to a full model!

In [ ]:
# model.save_adapter("./final_adapter", "sciie")

# !ls -lh final_adapter

**Share your work!**

The next step after training is to share our adapter with the world via _AdapterHub_. [Read our guide](https://docs.adapterhub.ml/contributing.html) on how to prepare the adapter module we just saved and contribute it to the Hub!

➡️ Also continue with [the next Colab notebook](https://colab.research.google.com/github/Adapter-Hub/adapter-transformers/blob/master/notebooks/02_Adapter_Inference.ipynb) to learn how to use adapters from the Hub.

In [ ]:
# from google.colab import files
# !zip -r /content/file.zip /content
# files.download("/content/file.zip")